In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import glob
import os
import netCDF4 as nc4
from scipy.interpolate import interp1d
import math
import matplotlib.tri as tri
#from plot_season import make_hist
import pickle as pkl
import sim_name_remap as map
import var_config as vc

In [2]:
sim_names = ['MAM4_CESM3_test']   #'MAM4_MEIC_SO2','MAM4_CAMS6.2_SO2','MAM4_MEIC_110L']#,'CARMA_MEIC_SO2_bins','CARMA_CAMS_SO2_bins']  #,    #  'MAM4_MEIC_SO2_ZMKE']  #,,]  ,
var_names = ['so4_a1','so4_a2','so4_a3','SO2']#'SO4']#,'SO2']#,'CO','O3']#,OA','BC','N2O','NOX','C2H6','C3H8','CFC11','CFC113','CH3CCL3','CCL4','CFC115','CFC12','CH2BR2','CH3BR','CH3CL',\
            #'CF2CLBR','HCFC141B','HCFC142B','HCFC22','CFC114','CF3BR','CH2CL2']
            #[,'soa1_a1','soa1_a2','soa2_a1','soa2_a2','soa3_a1','soa3_a2',\
            #'soa4_a1','soa4_a2','soa5_a1','soa5_a2','pom_a1','num_a1','num_a2','num_a3','dst_a1',\
            # 'dst_a2','dst_a3','bc_a1','ncl_a1','ncl_a2','ncl_a3']

lon_bndss = [[125,140]]    # [95,135],[125,140],[90,110]]#,[65,95],]]  ]#,    
lat_bndss = [[25,38]]    # [30,50],[25,38],[25,40]]#]#,,[30,40]]   
#lon_bndss = [[75,120]]
#lat_bndss = [[25,40]]

do_prof = 1  # Determines if the code will calculate mean profiles of model data to compare against aircraft
SAVEDIR = '/glade/derecho/scratch/wsmith/carma_vars/'
prs_lvls = [100.]   # This is used as a filter for screening in vs out of anticyclone
gph_asmas = [[16730,16810]]
z_prof = np.arange(0,25,0.5)  # Altitude profile (km)
th_prof = np.arange(300,600,5)  # Theta profile (K)
#delta_th_prof = np.arange(-100,101,5)
delta_z_prof = np.arange(-20,10.1,0.5) # Tropopause-relative altitude profile (km)
Rdcp = 287.06/1004.

In [3]:
for sim_name in sim_names:

    true_sim_path = map.remap[sim_name]
    print('True sim name: ', true_sim_path)

    print(true_sim_path)

    FILES = sorted(glob.glob(true_sim_path))
    #print(FILES)

    for ltln in range(len(lon_bndss)):

        lon_bnds = lon_bndss[ltln]
        lat_bnds = lat_bndss[ltln]

        # if true_sim_name[0:5] == 'CARMA': 
        #    DATAROOT = '/data/wsmith/forecasts/2022_ACCLIP_' + sim_name
        #    var_strs = [['MXSO4MR','SO4PRMR'],['MXOCMR','MXSOAMR'],['MXBCMR'],['CO'],['SO2'],['BIGALK'],['C2H6'],['C3H8'],['CH4'],['CH2O'],['H2O2'],['CH3OOH'],['NH3'],['OH']]                  
            
        for v in range(len(var_names)):
                            
            var_name = var_names[v]

            zSAVEFILE = SAVEDIR + var_name + 'zprofiles_' + sim_name + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '.pkl'

            if os.path.isfile(zSAVEFILE):
                print(var_name, ' files already exist, skipping them...')
            
            else: 

                if sim_name[0:4] == 'MAM4': 
                    if var_name in vc.var_info: var_str = vc.var_info[var_name]['MAM4_vars']
                    else: var_str = var_name
                if sim_name[0:5] == 'CARMA': var_str = vc.var_info[var_name]['CARMA_vars']
                
                #var_str = var_strs[v]
                #var_mod = var_mods[v]
                #rng = rngs[v]
    
                print(sim_name, '*****', var_str, '*****', lon_bnds, '*****', lat_bnds)
    
                file0 = nc4.Dataset(FILES[0])
    
                # Account for the fact that we might have two model variables that we need to combine
                if isinstance(var_str, str): var = file0.variables[var_str][:]
                if isinstance(var_str, list):
                    var = file0.variables[var_str[0]][:]
                    for sub_v in range(len(var_str)-1):
                        sub_var = file0.variables[var_str[sub_v+1]][:]
                        var = var + sub_var
                    
                temp   = file0.variables['T'][:]
                PS     = file0.variables['PS'][:]
                TROP_P = file0.variables['TROP_P'][:]   # This is not included in the output, but can be uncommented later for other simulations
                gph    = file0.variables['Z3'][:]
                #U      = file0.variables['U'][:]
                #V      = file0.variables['V'][:]
    
                for f in range(len(FILES)-1):
                    
                    file = nc4.Dataset(FILES[f+1])
                    
                    if isinstance(var_str, str): curr_var = file.variables[var_str][:]
                    if isinstance(var_str, list):
                        curr_var = file.variables[var_str[0]][:]
                        for sub_v in range(len(var_str)-1):
                            curr_sub_var = file.variables[var_str[sub_v+1]][:]
                            curr_var = curr_var + curr_sub_var
                    
                    var = np.append(var, curr_var, axis=0)
                    
                    temp   = np.append(temp, file.variables['T'][:], axis=0)
                    PS     = np.append(PS, file.variables['PS'][:], axis=0)
                    #U      = np.append(U, file.variables['U'][:], axis=0)   # We don't need U and V for this, they can be read in as their own variable
                    #V      = np.append(V, file.variables['V'][:], axis=0)
                    TROP_P = np.append(TROP_P, file.variables['TROP_P'][:], axis=0)
                    gph    = np.append(gph, file.variables['Z3'][:], axis=0)
                    
                #var = var*var_mod    # Moving this to the plotting script
                NT = np.shape(var)[0]
                
                PS_mean = np.mean(PS, axis=0)
    
                for p in range(len(prs_lvls)):
                
                    # Set up arrays so we can save the non-interpolated values
                    var_1d = []
                    troprelz_1d = []
                    z_1d = []
                    theta_1d = []
                    tropreltheta_1d = []
                
                    prs_lvl = prs_lvls[p]
                    gph_asma = gph_asmas[p]
                    TROP_gph_array = []
                    TROP_theta_array = []
    
                    lon  = file0.variables['lon'][:]
                    lat  = file0.variables['lat'][:]
                    lev  = file0.variables['lev'][:]
                    hyam = file0.variables['hyam'][:]
                    hybm = file0.variables['hybm'][:]
                    NZ   = len(lev)
    
                    var_lvls_f = interp1d(lev, np.arange(len(lev)), bounds_error=None, fill_value=-999)
                    var_lev_index_frac = var_lvls_f(prs_lvl)
                    var_lev_index_dec  = var_lev_index_frac - math.floor(var_lev_index_frac)
                    var_lev_index_flr  = math.floor(var_lev_index_frac)
    
                    var_zprofs_in  = np.zeros((1,len(z_prof)))-999
                    var_zprofs_out = np.zeros((1,len(z_prof)))-999
    
                    var_thprofs_in  = np.zeros((1,len(th_prof)))-999
                    var_thprofs_out = np.zeros((1,len(th_prof)))-999
                    
                    var_troprelzprofs_in  = np.zeros((1,len(delta_z_prof)))-999
                    var_troprelzprofs_out = np.zeros((1,len(delta_z_prof)))-999
                    
                    #var_troprelthprofs_in  = np.zeros((1,len(delta_th_prof)))-999
                    #var_troprelthprofs_out = np.zeros((1,len(delta_th_prof)))-999
                    
                    if len(lon) < 5000.:  # Structured grid
    
                        # Trim the plot to just the lon/lat boundaries
                        lon_ind0 = np.min(np.where(lon >= lon_bnds[0])[0])
                        lon_ind1 = np.max(np.where(lon <= lon_bnds[1])[0])+1
                        lat_ind0 = np.min(np.where(lat >= lat_bnds[0])[0])
                        lat_ind1 = np.max(np.where(lat <= lat_bnds[1])[0])+1
                        
                        var_sub = var[:,:,lat_ind0:lat_ind1,lon_ind0:lon_ind1]
                        TROP_P_sub = TROP_P[:,lat_ind0:lat_ind1,lon_ind0:lon_ind1]
                        PS_sub = PS[:,lat_ind0:lat_ind1,lon_ind0:lon_ind1]
                        temp_sub = temp[:,:,lat_ind0:lat_ind1,lon_ind0:lon_ind1]
                        gph_sub = gph[:,:,lat_ind0:lat_ind1,lon_ind0:lon_ind1]
                        var_lev = (1-var_lev_index_dec)*var_sub[:,var_lev_index_flr,:,:] + var_lev_index_dec*var_sub[:,var_lev_index_flr+1,:,:]   
                        gph_lev = (1-var_lev_index_dec)*gph_sub[:,var_lev_index_flr,:,:] + var_lev_index_dec*gph_sub[:,var_lev_index_flr+1,:,:] 
                        #U_lev = (1-var_lev_index_dec)*U[:,var_lev_index_flr,lat_ind0:lat_ind1,lon_ind0:lon_ind1] + var_lev_index_dec*U[:,var_lev_index_flr+1,lat_ind0:lat_ind1,lon_ind0:lon_ind1] 
                        #V_lev = (1-var_lev_index_dec)*V[:,var_lev_index_flr,lat_ind0:lat_ind1,lon_ind0:lon_ind1] + var_lev_index_dec*V[:,var_lev_index_flr+1,lat_ind0:lat_ind1,lon_ind0:lon_ind1] 
                        
                        lon_sub = lon[lon_ind0:lon_ind1]
                        lat_sub = lat[lat_ind0:lat_ind1]
                        
                        if do_prof:
                        
                            ################ Compute average profiles within and outside the anticyclone
                            for t in range(NT):
                                #print('****************  t = ', t)
    
                                # At each time step, we will generate a new array of profiles so we can add that to the master array all at one time
                                curr_var_zprofs_in  = np.zeros((1,len(z_prof)))-999
                                curr_var_zprofs_out = np.zeros((1,len(z_prof)))-999
    
                                curr_var_thprofs_in  = np.zeros((1,len(th_prof)))-999
                                curr_var_thprofs_out = np.zeros((1,len(th_prof)))-999
                                
                                curr_var_troprelzprofs_in  = np.zeros((1,len(delta_z_prof)))-999
                                curr_var_troprelzprofs_out = np.zeros((1,len(delta_z_prof)))-999
    
                                #curr_var_troprelthprofs_in  = np.zeros((1,len(delta_th_prof)))-999
                                #curr_var_troprelthprofs_out = np.zeros((1,len(delta_th_prof)))-999
                            
                                lt_in = []
                                ln_in = []
                                lt_out = []
                                ln_out = []
                                
                                for lt in range(len(lat_sub)):
                                    for ln in range(len(lon_sub)):
                                
                                        # Unfortunately these simulations are on "raw" model levels which means we need to account for topography.  
                                        curr_TROP_P = TROP_P_sub[t,lt,ln]/100.
                                        curr_PS = PS_sub[t,lt,ln] 
                                        curr_plev = (hyam*1.0e5 + hybm*curr_PS)/100.
                                        curr_gph_lev = gph_lev[t,lt,ln]                            
                                        curr_gphprof = np.squeeze(gph_sub[t,:,lt,ln])                            
                                        curr_varprof = np.squeeze(var_sub[t,:,lt,ln])
                                        
                                        plev_f = interp1d(curr_plev, np.arange(len(curr_plev)), bounds_error=False, fill_value=-999)
                                        trop_index_frac = plev_f(curr_TROP_P)
                                        gph_f = interp1d(np.arange(len(curr_gphprof)), curr_gphprof, bounds_error=False, fill_value=-999)
                                        curr_TROP_gph = gph_f(trop_index_frac)
                                        TROP_gph_array.append(curr_TROP_gph)
    
                                        curr_deltazlev = (curr_gphprof - curr_TROP_gph)/1.0e3
                                        f_z = interp1d(curr_gphprof*1.e-3, curr_varprof, bounds_error=False, fill_value=-999)
                                        f_deltaz = interp1d(curr_deltazlev, curr_varprof, bounds_error=False, fill_value=-999)                                                                                                                                  
                                        
                                        curr_tempprof = np.squeeze(temp_sub[t,:,lt,ln])
                                        curr_thetaprof = curr_tempprof*((1000./curr_plev)**Rdcp)
                                        f_th = interp1d(curr_thetaprof, curr_varprof, bounds_error=False, fill_value=-999)
    
                                        #th_f = interp1d(np.arange(len(curr_thetaprof)), curr_thetaprof, bounds_error=False, fill_value=-999)
                                        #curr_TROP_theta = th_f(trop_index_frac)  
                                        #TROP_theta_array.append(curr_TROP_theta)
                                        #curr_TROP_theta = curr_thetaprof[TROP_nrst_index]  # Current tropopause in theta coordinates
                                        #curr_deltathetaprof = curr_thetaprof - curr_TROP_theta
                                        #f_deltatheta = interp1d(curr_deltathetaprof, curr_varprof, bounds_error=False, fill_value=-999)  # The 9s are intentional to flag missing data as negative (missing)                                          
    
                                        var_1d.extend(curr_varprof)
                                        z_1d.extend(curr_gphprof*1.e-3)
                                        #troprelz_1d.extend(curr_deltazlev)
                                        theta_1d.extend(curr_thetaprof)
                                        #tropreltheta_1d.extend(curr_deltathetaprof)
    
                                        if curr_gph_lev >= np.mean(gph_asma): 
                                            #ln_in.append(ln)
                                            #lt_in.append(lt)
                                            #var_profs_in_theta  = np.append(var_profs_in_theta, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            curr_var_zprofs_in = np.append(curr_var_zprofs_in, f_z(z_prof)[None,:], axis=0)
                                            curr_var_thprofs_in = np.append(curr_var_thprofs_in, f_th(th_prof)[None,:], axis=0)
                                            #curr_var_troprelthprofs_in = np.append(curr_var_troprelthprofs_in, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            curr_var_troprelzprofs_in  = np.append(curr_var_troprelzprofs_in, f_deltaz(delta_z_prof)[None,:], axis=0)
                                        else:
                                            #ln_out.append(ln)
                                            #lt_out.append(lt)
                                            #var_profs_out_theta = np.append(var_profs_out_theta, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            curr_var_zprofs_out  = np.append(curr_var_zprofs_out, f_z(z_prof)[None,:], axis=0)    
                                            curr_var_thprofs_out = np.append(curr_var_thprofs_out, f_th(th_prof)[None,:], axis=0)                                    
                                            #curr_var_troprelthprofs_out = np.append(curr_var_troprelthprofs_out, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            curr_var_troprelzprofs_out  = np.append(curr_var_troprelzprofs_out, f_deltaz(delta_z_prof)[None,:], axis=0)                                                                
                                
                                var_zprofs_in = np.append(var_zprofs_in, curr_var_zprofs_in[1:,:], axis=0)
                                var_zprofs_out = np.append(var_zprofs_out, curr_var_zprofs_out[1:,:], axis=0)                        
    
                                var_thprofs_in = np.append(var_thprofs_in, curr_var_thprofs_in[1:,:], axis=0)
                                var_thprofs_out = np.append(var_thprofs_out, curr_var_thprofs_out[1:,:], axis=0) 
                                
                                var_troprelzprofs_in = np.append(var_troprelzprofs_in, curr_var_troprelzprofs_in[1:,:], axis=0)
                                var_troprelzprofs_out = np.append(var_troprelzprofs_out, curr_var_troprelzprofs_out[1:,:], axis=0)
    
                                #var_troprelthprofs_in  = np.append(var_troprelthprofs_in, curr_var_troprelthprofs_in[1:,:], axis=0)
                                #var_troprelthprofs_out = np.append(var_troprelthprofs_out, curr_var_troprelthprofs_out[1:,:], axis=0)
                                
                                #var_profs_in  = np.append(var_profs_in, var_sub[t,:,lt_in,ln_in], axis=0)
                                #var_profs_out = np.append(var_profs_out, var_sub[t,:,lt_out,ln_out], axis=0)
                        
                            #print(np.shape(var_zprofs_in))
                            #print(np.shape(var_zprofs_out))
                            
                        PS_plot = np.mean(PS_sub, axis=0)
                    
                    else:  # Regional refinement
    
                        print('This is a regionally refined grid - converting to rectangular grid...')
                        xi = np.linspace(0.0, 359.75, 1440)  # For now, we define a quarter degree rectangular grid to interpolate to
                        yi = np.linspace(-90, 90, 721)
                        triang = tri.Triangulation(lon, lat)
                        Xi, Yi = np.meshgrid(xi, yi)    
                        
                        #var_lev = np.zeros((1, len(yi), len(xi)))
                        #gph_lev = np.zeros((1, len(yi), len(xi)))
                        #U_lev = np.zeros((1, len(yi), len(xi)))
                        #V_lev = np.zeros((1, len(yi), len(xi)))
                        
                        PS_interpolator = tri.LinearTriInterpolator(triang, PS_mean)   
                        PS_plot = PS_interpolator(Xi, Yi)
                            
                        for t in range(NT):
                            #print('t = ', t)
                            
                            #var_arr = np.squeeze((1-var_lev_index_dec)*var[t,var_lev_index_flr,:] + var_lev_index_dec*var[t,var_lev_index_flr+1,:])
                            #var_interpolator = tri.LinearTriInterpolator(triang, var_arr)   
                            #var_lev = np.append(var_lev, var_interpolator(Xi, Yi)[None,:], axis=0)
                            
                            gph_arr = np.squeeze((1-var_lev_index_dec)*gph[t,var_lev_index_flr,:] + var_lev_index_dec*gph[t,var_lev_index_flr+1,:])
                            #gph_interpolator = tri.LinearTriInterpolator(triang, gph_arr)
                            #gph_lev = np.append(gph_lev, gph_interpolator(Xi, Yi)[None,:], axis=0)
    
                            #U_arr = np.squeeze((1-var_lev_index_dec)*U[t,var_lev_index_flr,:] + var_lev_index_dec*U[t,var_lev_index_flr+1,:])
                            #U_interpolator = tri.LinearTriInterpolator(triang, U_arr)
                            #U_lev = np.append(U_lev, U_interpolator(Xi, Yi)[None,:], axis=0)
    
                            #V_arr = np.squeeze((1-var_lev_index_dec)*V[t,var_lev_index_flr,:] + var_lev_index_dec*V[t,var_lev_index_flr+1,:])
                            #V_interpolator = tri.LinearTriInterpolator(triang, V_arr)   
                            #V_lev = np.append(V_lev, V_interpolator(Xi, Yi)[None,:], axis=0)
                            
                            pts_in = []  # For each time step, we will save the points inside and outside the monsoon
                            pts_out = []
    
                            # At each time step, we will generate a new array of profiles so we can add that to the master array all at one time
                            curr_var_zprofs_in  = np.zeros((1,len(z_prof)))-999
                            curr_var_zprofs_out = np.zeros((1,len(z_prof)))-999
    
                            curr_var_thprofs_in  = np.zeros((1,len(th_prof)))-999
                            curr_var_thprofs_out = np.zeros((1,len(th_prof)))-999
                            
                            #curr_var_troprelzprofs_in  = np.zeros((1,len(delta_z_prof)))-999
                            #curr_var_troprelzprofs_out = np.zeros((1,len(delta_z_prof)))-999
    
                            #curr_var_troprelthprofs_in  = np.zeros((1,len(delta_th_prof)))-999
                            #curr_var_troprelthprofs_out = np.zeros((1,len(delta_th_prof)))-999
                            
                            if do_prof:
                            
                                for pt in range(len(lon)):
                                
                                    if pt % 10000 == 0: print(pt, ' of ', len(lon))
                                   
                                    curr_lon = lon[pt]
                                    curr_lat = lat[pt]
                                    
                                    if curr_lon >= lon_bnds[0] and curr_lon <= lon_bnds[1] and curr_lat >= lat_bnds[0] and curr_lat <= lat_bnds[1]:
    
                                        # Unfortunately these simulations are on "raw" model levels which means we need to account for topography.  
                                        curr_TROP_P = TROP_P[t,pt]/100.
                                        curr_PS = PS[t,pt] 
                                        curr_plev = (hyam*1.0e5 + hybm*curr_PS)/100.
                                        curr_gph_lev = gph_arr[pt]       
                                        curr_gphprof = np.squeeze(gph[t,:,pt])                            
                                        curr_varprof = np.squeeze(var[t,:,pt])
                                        
                                        plev_f = interp1d(curr_plev, np.arange(len(curr_plev)), bounds_error=False, fill_value=-999)
                                        trop_index_frac = plev_f(curr_TROP_P)
                                        gph_f = interp1d(np.arange(len(curr_gphprof)), curr_gphprof, bounds_error=False, fill_value=-999)
                                        curr_TROP_gph = gph_f(trop_index_frac)
                                        TROP_gph_array.append(curr_TROP_gph)
                                        
                                        #curr_deltazlev = (curr_gphprof - curr_TROP_gph)/1.0e3
                                        f_z = interp1d(curr_gphprof*1.e-3, curr_varprof, bounds_error=False, fill_value=-999)
                                        f_deltaz = interp1d(curr_deltazlev, curr_varprof, bounds_error=False, fill_value=-999)  
                                     
                                        curr_tempprof = np.squeeze(temp[t,:,pt])                        
                                        curr_thetaprof = curr_tempprof*((1000./curr_plev)**Rdcp) 
                                        f_th = interp1d(curr_thetaprof, curr_varprof, bounds_error=False, fill_value=-999)
                                        
                                        th_f = interp1d(np.arange(len(curr_thetaprof)), curr_thetaprof, bounds_error=False, fill_value=-999)
                                        ##curr_TROP_theta = th_f(trop_index_frac)
                                        #TROP_theta_array.append(curr_TROP_theta)                            
                                        #curr_TROP_theta = curr_thetaprof[TROP_nrst_index]  # Current tropopause in theta coordinates
                                        #curr_deltathetaprof = curr_thetaprof - curr_TROP_theta
                                        #f_deltatheta = interp1d(curr_deltathetaprof, curr_varprof, bounds_error=False, fill_value=-999)  # The 9s are intentional to flag data below the surface as negative
    
                                        curr_gph = gph_arr[pt]
                                        
                                        var_1d.extend(curr_varprof)
                                        z_1d.extend(curr_gphprof*1.e-3)
                                        #troprelz_1d.extend(curr_deltazlev)
                                        theta_1d.extend(curr_thetaprof)
                                        #tropreltheta_1d.extend(curr_deltathetaprof)
                                        
                                        if curr_gph >= np.mean(gph_asma): 
                                            #pts_in.append(pt)
                                            #var_profs_in_theta  = np.append(var_profs_in_theta, f_deltatheta(delta_theta_prof)[None,:], axis=0) 
                                            curr_var_zprofs_in = np.append(curr_var_zprofs_in, f_z(z_prof)[None,:], axis=0)
                                            curr_var_thprofs_in = np.append(curr_var_thprofs_in, f_th(th_prof)[None,:], axis=0)
                                            #curr_var_troprelthprofs_in = np.append(curr_var_troprelthprofs_in, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            #curr_var_troprelzprofs_in  = np.append(curr_var_troprelzprofs_in, f_deltaz(delta_z_prof)[None,:], axis=0)                                
                                        else: 
                                            #pts_out.append(pt)
                                            #var_profs_out_theta = np.append(var_profs_out_theta, f_deltatheta(delta_theta_prof)[None,:], axis=0)
                                            curr_var_zprofs_out = np.append(curr_var_zprofs_out, f_z(z_prof)[None,:], axis=0)
                                            curr_var_thprofs_out = np.append(curr_var_thprofs_out, f_th(th_prof)[None,:], axis=0)
                                            #curr_var_troprelthprofs_out = np.append(curr_var_troprelthprofs_out, f_deltatheta(delta_th_prof)[None,:], axis=0)
                                            #curr_var_troprelzprofs_out  = np.append(curr_var_troprelzprofs_out, f_deltaz(delta_z_prof)[None,:], axis=0)                    
    
                                var_zprofs_in = np.append(var_zprofs_in, curr_var_zprofs_in[1:,:], axis=0)
                                var_zprofs_out = np.append(var_zprofs_out, curr_var_zprofs_out[1:,:], axis=0)  
    
                                var_thprofs_in = np.append(var_thprofs_in, curr_var_thprofs_in[1:,:], axis=0)
                                var_thprofs_out = np.append(var_thprofs_out, curr_var_thprofs_out[1:,:], axis=0)
                                
                                #var_troprelzprofs_in = np.append(var_troprelzprofs_in, curr_var_troprelzprofs_in[1:,:], axis=0)
                                #var_troprelzprofs_out = np.append(var_troprelzprofs_out, curr_var_troprelzprofs_out[1:,:], axis=0)
    
                                #var_troprelthprofs_in  = np.append(var_troprelthprofs_in, curr_var_troprelthprofs_in[1:,:], axis=0)
                                #var_troprelthprofs_out = np.append(var_troprelthprofs_out, curr_var_troprelthprofs_out[1:,:], axis=0)
                                                                                                                                  
                        #var_lev = var_lev[1:,:,:]
                        #gph_lev = gph_lev[1:,:,:]
                        #U_lev = U_lev[1:,:,:]
                        #V_lev = V_lev[1:,:,:]
                        
                        lon = xi
                        lat = yi
                        
                        lon_sub = lon
                        lat_sub = lat
                        #lon_ind0 = np.min(np.where(lon >= lon_bnds[0])[0])
                        #lon_ind1 = np.max(np.where(lon <= lon_bnds[1])[0])+1
                        #lat_ind0 = np.min(np.where(lat >= lat_bnds[0])[0])
                        #lat_ind1 = np.max(np.where(lat <= lat_bnds[1])[0])+1
    
                    #var_lev_mean = np.mean(var_lev, axis=0)
                    #gph_lev_mean = np.mean(gph_lev, axis=0)
                    #U_lev_mean = np.mean(U_lev, axis=0)
                    #V_lev_mean = np.mean(V_lev, axis=0)       
                    
                    
                    ################# Save the profile information
                    
                    #with open(SAVEDIR + var_name + 'profiles_' + sim + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '_' + str(int(prs_lvl)) + 'GPHfilter.pkl', 'wb') as f:
                    #    pkl.dump((var_profs_in[1:,:], var_profs_out[1:,:], lev, gph_asma), f)
    
                    if do_prof:
    
                        with open(zSAVEFILE, 'wb') as f:
                            pkl.dump((np.append(var_zprofs_in[1:,:], var_zprofs_out[1:,:], axis=0), TROP_gph_array, z_prof), f)
    
                        with open(SAVEDIR + var_name + 'thetaprofiles_' + sim_name + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '.pkl', 'wb') as f:
                            pkl.dump((np.append(var_thprofs_in[1:,:], var_thprofs_out[1:,:], axis=0), TROP_theta_array, th_prof), f)
                            
                        #with open(SAVEDIR + var_name + 'troprelthetaprofiles_' + sim + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '_' + str(int(prs_lvl)) + 'GPHfilter.pkl', 'wb') as f:
                        #    pkl.dump((var_troprelthprofs_in[1:,:], var_troprelthprofs_out[1:,:], TROP_theta_array, delta_th_prof, gph_asma), f)
    
                        with open(SAVEDIR + var_name + 'troprelzprofiles_' + sim_name + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '.pkl', 'wb') as f:
                            pkl.dump((np.append(var_troprelzprofs_in[1:,:], var_troprelzprofs_out[1:,:], axis=0), TROP_gph_array, delta_z_prof), f)
                            
                        # Also save the non-interpolated values
                        with open(SAVEDIR + var_name + '_1d_nointerpvals_' + sim_name + '_' + str(lon_bnds) + '_' + str(lat_bnds) + '.pkl', 'wb') as f:
                            pkl.dump((var_1d, z_1d, troprelz_1d, theta_1d, tropreltheta_1d), f)
                        

True sim name:  /glade/derecho/scratch/tilmes/archive/f.e30.alpha07e.ctsm5.3.075.sp.nudged.2016_2018_MTt4s.ne30.002/atm/hist/f.e30*.h0a.2022-08*
/glade/derecho/scratch/tilmes/archive/f.e30.alpha07e.ctsm5.3.075.sp.nudged.2016_2018_MTt4s.ne30.002/atm/hist/f.e30*.h0a.2022-08*
so4_a1  files already exist, skipping them...
so4_a2  files already exist, skipping them...
so4_a3  files already exist, skipping them...
MAM4_CESM3_test ***** ['SO2'] ***** [125, 140] ***** [25, 38]
